# Segmenting and Clustering Neighborhoods in Toronto

This Notebook builds the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the neighborhoods in Toronto in a form of DataFrame


In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

## 1. Scrape the Wikipedia page and convert data into a DataFrame

Find the first table in the webpage, scrape all the data and store data in a DataFrame

In [2]:
source  = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
neigh = soup.find_all('table')[0]
new_table = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])

temp = pd.DataFrame([('','','')])
temp.columns = ['PostalCode', 'Borough', 'Neighborhood']

for row in neigh.find_all('tr'):
    j = 0
    columns = row.find_all('td')
    for column in columns:
        temp.iloc[0,j] = column.get_text().strip()
        j+=1
    new_table = new_table.append(temp)

#remove the first redandunt row
new_table = new_table[1:]
new_table.reset_index(drop=True, inplace=True)

## 2. Clean the data

Remove row where Boroughs are "Not assigned". If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [3]:
# remove rows where Borough not assigned, reset index after it
new_table = new_table[new_table.Borough != 'Not assigned']
new_table.sort_values(by=['PostalCode'], axis=0, inplace=True)
new_table.reset_index(drop=True, inplace=True)

# if Neighborhood is unassigned, set it to Borough
not_assigned = new_table[new_table['Neighborhood']=="Not assigned"]
new_table.iloc[not_assigned.index, 2] = new_table.iloc[not_assigned.index, 1]
new_table

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek
5,M1E,Scarborough,Guildwood
6,M1E,Scarborough,Morningside
7,M1E,Scarborough,West Hill
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


## 3. Build the structure in accordance with the instruction

Find the unique Postal Codes and then combine the Neighborhood if Borough is the same 

In [4]:
neighborhoods = new_table.groupby(['PostalCode', 'Borough']).size().reset_index(name='Neighborhood')

#for matching
codes = new_table['PostalCode'].unique().tolist()

for i in range(len(neighborhoods)):
    cut = new_table[new_table['PostalCode']==codes[i]]
    neighborhoods.iloc[i,2] = cut.iloc[0,2]
    for j in range(len(cut)-1):
        neighborhoods.iloc[i,2] = neighborhoods.iloc[i,2] + ', ' + cut.iloc[j+1,2]

print(neighborhoods)

    PostalCode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
16         M1X       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
21         M2M        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North 

In [5]:
print(neighborhoods.shape)

(103, 3)


## 4. Download the geographical coordinates of a given postal code

In [6]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

# All requested packages already installed.



The while loop does not stop. Read the geographical coordinates from the .csv file directly

In [7]:
"""
import geocoder # import geocoder
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates

while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude, longitude)
"""

"\nimport geocoder # import geocoder\n# initialize your variable to None\nlat_lng_coords = None\n\n# loop until you get the coordinates\n\nwhile(lat_lng_coords is None):\n  g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))\n  lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]\nprint(latitude, longitude)\n"

## 5. Add the geographical coordinates of the neighborhoods by using the downloaded file

Use merge and inner join to add the latitude and longitude

In [8]:
coordinates_df = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates_df.head()
coordinates_df.rename(columns={'Postal Code': 'PostalCode'}, inplace = True)
coordinates_df.head()
neighborhoods = pd.merge(neighborhoods, coordinates_df, on='PostalCode', how='inner')
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


## 6. Start segmenting and clustering neighborhoods 

Download the required libraries

In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


Use geopy library to get the latitude and longitude values of Toronta

In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronta are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronta are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [11]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## 7. Start exploring neighborhoods 

Define Foursquare Credentials and Version

In [12]:
# The code was removed by Watson Studio for sharing.

Explore neighborhoods in our dataframe by getting the top 100 venues that are in neighborhoods within a radius of 500 meters

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
                
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
        
    return(nearby_venues)   

For some reason, 3 neighborhoods are not returned

In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
Toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Rouge, Malvern
Port Union, Rouge Hill, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Golden Mile, Oakridge, Clairlea
Cliffcrest, Scarborough Village West, Cliffside
Cliffside West, Birch Cliff
Wexford Heights, Dorset Park, Scarborough Town Centre
Maryvale, Wexford
Agincourt
Sullivan, Clarks Corners, Tam O'Shanter
Milliken, Agincourt North, L'Amoreaux East, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Oriole, Henry Farm
Bayview Village
Silver Hills, York Mills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South, Flemingdon Park
Wilson Heights, Downsview North, Bathurst Manor
Northwood Park, York University
Downsview East, CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [15]:
print(Toronto_venues.shape)
print(Toronto_venues.head())
print(Toronto_venues.groupby('Neighborhood').count().head(5))
print('\nThere are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

(2246, 7)
                             Neighborhood  Neighborhood Latitude  \
0                          Rouge, Malvern              43.806686   
1                          Rouge, Malvern              43.806686   
2  Port Union, Rouge Hill, Highland Creek              43.784535   
3       Guildwood, Morningside, West Hill              43.763573   
4       Guildwood, Morningside, West Hill              43.763573   

   Neighborhood Longitude                            Venue  Venue Latitude  \
0              -79.194353                          Wendy's       43.807448   
1              -79.194353            Interprovincial Group       43.805630   
2              -79.160497            Royal Canadian Legion       43.782533   
3              -79.188711  Swiss Chalet Rotisserie & Grill       43.767697   
4              -79.188711                G & G Electronics       43.765309   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.200378        

In [16]:
Toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection


In [17]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, South Steeles, Thistletown, Silverstone",9,9,9,9,9,9
Bayview Village,4,4,4,4,4,4
Berczy Park,55,55,55,55,55,55
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"Cabbagetown, St. James Town",47,47,47,47,47,47
Caledonia-Fairbanks,6,6,6,6,6,6
Canada Post Gateway Processing Centre,11,11,11,11,11,11
Cedarbrae,8,8,8,8,8,8


## 8. Analyze each neighborhood

Each row is one venue

In [18]:
# one hot encoding
#the venue category of index 436,312,1177,1093 are "neighborhood"
#print(Toronto_venues[['Venue Category']].sort_values(by='Venue Category',axis=0))
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe. Have to renamed it to "Neighborhoods" as there is a venue cateogry called "Neighborhood"
toronto_onehot['Neighborhoods'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2246, 282)


,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,War

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
print(toronto_grouped.head)
print(toronto_grouped.shape)

<bound method NDFrame.head of                                         Neighborhoods  Accessories Store  \
0                                           Agincourt                0.0   
1   Albion Gardens, Beaumond Heights, Humbergate, ...                0.0   
2                                     Bayview Village                0.0   
3                                         Berczy Park                0.0   
4   Business Reply Mail Processing Centre 969 Eastern                0.0   
5                         Cabbagetown, St. James Town                0.0   
6                                 Caledonia-Fairbanks                0.0   
7               Canada Post Gateway Processing Centre                0.0   
8                                           Cedarbrae                0.0   
9                                  Central Bay Street                0.0   
10                                           Christie                0.0   
11                               Church and Wellesley     

Print each neighborhood along with the top 5 most common venues

In [20]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    # do not need the first title row
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2      Sandwich Place   0.2
3  Chinese Restaurant   0.2
4      Clothing Store   0.2


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, South Steeles, Thistletown, Silverstone----
                  venue  freq
0         Grocery Store  0.22
1   Fried Chicken Joint  0.11
2              Pharmacy  0.11
3           Pizza Place  0.11
4  Fast Food Restaurant  0.11


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4    Accessories Store  0.00


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2   Cheese Shop  0.04
3        Bakery  0.04
4      Beer Bar  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.12
1           Yoga Studio  0.06
2         

## 9. Put that into a pandas dataframe

Write a function to sort the venues in descending order.

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted[60:70]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,Queen's Park,Coffee Shop,Gym,Japanese Restaurant,Sushi Restaurant,Park,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place
61,"Railway Lands, South Niagara, Harbourfront Wes...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Airport Gate,Plane,Sculpture Garden,Airport Food Court,Boutique,Harbor / Marina
62,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Bank
63,"Richview Gardens, Kingsview Village, St. Phill...",Mobile Phone Shop,Park,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
64,"Roncesvalles, Parkdale",Gift Shop,Breakfast Spot,Italian Restaurant,Eastern European Restaurant,Bar,Bank,Dog Run,Restaurant,Movie Theater,Dessert Shop
65,Rosedale,Park,Playground,Trail,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
66,Roselawn,Garden,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Dim Sum Restaurant
67,"Rouge, Malvern",Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
68,"Runnymede, Swansea",Café,Pizza Place,Coffee Shop,Gym,Italian Restaurant,Sushi Restaurant,Sandwich Place,Indie Movie Theater,French Restaurant,Diner
69,"Ryerson, Garden District",Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Pizza Place,Italian Restaurant,Bubble Tea Shop,Fast Food Restaurant


## 10. Put that into a pandas dataframe
Run k-means to cluster the neighborhood into 5 clusters.

In [23]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood. Need to use how=right to make sure the cluster label is integer
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497,2,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Diner
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Intersection,Spa,Rental Car Location,Electronics Store,Breakfast Spot,Mexican Restaurant,Pizza Place,Medical Center,Ethiopian Restaurant,Empanada Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Bakery,Hakka Restaurant,Caribbean Restaurant,Athletics & Sports,Bank,Lounge,Thai Restaurant,Fried Chicken Joint,Yoga Studio,Drugstore


Visualize the resulting clusters

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
#len(ys) is the number of color
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 11. Examine Clusters
Examine each cluster and determine the discriminating venue categories that distinguish each cluster

### Cluster 1 (Has lots of baseball field)

In [25]:
print(toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].shape)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

(2, 12)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,1,Baseball Field,Construction & Landscaping,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
97,North York,1,Baseball Field,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Diner


### Cluster 2 (Has lots of restaurants and coffee shops)

In [26]:
print(toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].shape)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

(85, 12)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,Scarborough,2,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Diner
2,Scarborough,2,Intersection,Spa,Rental Car Location,Electronics Store,Breakfast Spot,Mexican Restaurant,Pizza Place,Medical Center,Ethiopian Restaurant,Empanada Restaurant
3,Scarborough,2,Coffee Shop,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
4,Scarborough,2,Bakery,Hakka Restaurant,Caribbean Restaurant,Athletics & Sports,Bank,Lounge,Thai Restaurant,Fried Chicken Joint,Yoga Studio,Drugstore
5,Scarborough,2,Playground,Cosmetics Shop,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
6,Scarborough,2,Hobby Shop,Convenience Store,Department Store,Coffee Shop,Bus Station,Discount Store,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop
7,Scarborough,2,Bus Line,Bakery,Soccer Field,Fast Food Restaurant,Park,Intersection,Metro Station,Bus Station,Doner Restaurant,Donut Shop
8,Scarborough,2,Motel,American Restaurant,Yoga Studio,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
9,Scarborough,2,College Stadium,Skating Rink,General Entertainment,Café,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant


### Cluster 3 (Has lots of garden)

In [27]:
print(toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].shape)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

(1, 12)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Garden,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Dim Sum Restaurant


### Cluster 4 (Has lot of banks)

In [28]:
print(toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].shape)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

(1, 12)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,4,Bank,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Diner


### Cluster 5 (nothing)

In [29]:
print(toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].shape)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

(0, 12)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
